Some $\LaTeX$ abbreviations:
$$
\newcommand{\pd}[2]{\frac{\partial #1}{\partial #2}}
$$

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [3]:
import astropy.cosmology as cosmo
from astropy.cosmology import Planck15
import astropy.units as u
import h5py

Let's try to estimate the precision of our measurement of $w$ by "collapsing" our 1000s of measurements of mass and distances into a single effective measurement at a "pivot" redshift of $z \sim 0.5$.  We have 
$$
d_L\left( z \mid H_0, \Omega_M, w \right) = \left( 1 + z \right) \int_0^z \mathrm{d} z' \, \frac{c}{H\left( z' \mid H_0, \Omega_M, w \right)},
$$
where 
$$
H\left( z' \mid H_0, \Omega_M, w \right) = H_0 \sqrt{\Omega_M \left( 1+z\right)^3 + \left(1 - \Omega_M \right) \left( 1 + z \right)^{3(1+w)}} \equiv H_0 E\left( z \mid \Omega_M, w \right).
$$

We will approximate 
$$
\sigma_w \simeq \left( \pd{\ln d_L}{w} \right)^{-1} \sqrt{ \sigma_{\ln d_L}^2  + \left( \pd{\ln d_L}{z} \right)^2 \sigma_z^2 + \left( \pd{\ln d_L}{\ln H_0} \right)^2 \sigma_{\ln H_0}^2 + \left( \pd{\ln d_L}{\Omega_M} \right)^2 \sigma_{\Omega_M}^2 }
$$
where $\sigma_{\ln d_L}$ and $\sigma_z \simeq \sigma_{\ln m_1}$ come from the measurement accuracy of the observations, either by combining the uncertainties from all the measurements via 
$$
\sigma_x^2 = \left[ \sum_{i} \left(\sigma_x^{(i)}\right)^{-2} \right]^{-1}
$$
or by taking a typical measurement uncertainty and scaling by $1/\sqrt{N}$, and $\sigma_{\ln H_0}$ and $\sigma_{\Omega_M}$ are the uncertainties in the prior measurements of these quantities ($\sim 1\%$ in both cases).

The partials above should be computed at a fiducial value of the parameters, say $H_0 = 70 \, \mathrm{km} \, \mathrm{s}^{-1} \, \mathrm{Mpc}$, $\Omega_M = 0.3$, $w = -1$ (to leading order the uncertainty budget is independent of these parameters).  Below we compute them.

In [17]:
z_pivot = 0.5
dlndLdz = 1/(1+z_pivot) + (1+z_pivot)*Planck15.hubble_distance/Planck15.luminosity_distance(z_pivot)
print(dlndLdz)

2.9414005273015023


$$
\pd{\ln d_L}{z} = \frac{1}{1+z} + \frac{\left(1+z\right) d_H}{d_L} \simeq 2.9
$$

The next one is simple:

$$
\pd{\ln d_L}{\ln H_0} \equiv -1
$$

In [18]:
zs = expm1(linspace(log(1), log(1+z_pivot), 100))
Ezs = Planck15.efunc(zs)
numer = trapz(((1+zs)**3-1)/(2*Ezs**3), zs)
denom = trapz(1/Ezs, zs)
dlndLdOm = numer/denom
print(dlndLdOm)

0.34337926212429837


$$
\pd{\ln d_L}{\Omega_M} = \frac{\int_0^z \mathrm{d}z' \, \frac{\left( 1 + z' \right)^3 - \left(1+z'\right)^{3(1+w)}}{2 E^{3}\left(z'\right)}}{\int_0^z \mathrm{d}z' \, \frac{1}{E\left(z'\right)}} \simeq 0.34
$$

In [19]:
numer = trapz(3*(1-0.3)*log1p(zs)/(2*Ezs**3), zs)
dlndLdw=numer/denom
print(dlndLdw)

0.15621504604988443


$$
\pd{\ln d_L}{w} = \frac{\int_0^z \mathrm{d} z' \, \frac{3 \left( 1 - \Omega_M \right) \ln\left( 1 + z'\right) \left( 1 + z' \right)^{3(1+w)}}{2 E^3\left( z' \right)}}{\int_0^z \mathrm{d} z' \frac{1}{E\left( z' \right)}} \simeq 0.16
$$

Now we just need to compute the $\sigma$s.  The only tricky bit is that only those masses that are sufficiently close to the upper boundary contribute.  For the moment let's leave that out, and just estimate.

In [16]:
with h5py.File('observations.h5', 'r') as f:
    sigma_lndl = 1.0/sqrt(sum(1/var(log(f['posteriors/dl']), axis=1)))
    sigma_z = 1.0/sqrt(sum(1/var(log(f['posteriors/m1det']), axis=1)))
sigma_lnH0 = 0.01
sigma_Om = 0.01

If we put it all together, we estimate a 7.5% uncertainty---not bad!

In [20]:
1/dlndLdw*sqrt(sigma_lndl**2 + dlndLdz**2*sigma_z**2 + sigma_lnH0**2 + dlndLdOm**2*sigma_Om**2)

<Quantity 0.07408415>

And if we assume we knew $H_0$ and $\Omega_M$ perfectly, the limit on $w$ would be about 3%.  Neat-o!

In [22]:
1/dlndLdw*sqrt(sigma_lndl**2 + dlndLdz**2*sigma_z**2)

<Quantity 0.03012399>